# Prepare

In [1]:
from openai import OpenAI
import importlib
import pandas as pd
from collections import deque
from pprint import pprint
import os
import hparams as hp
importlib.reload(hp)
from utils import num_tokens_from_messages, list_files, generate_file_content_string

prompt_history_queue = deque(maxlen=hp.PROMPT_HISTORY_LEN * 2)
client = OpenAI(api_key=hp.OPENAI_API_KEY)

In [2]:
def get_project_info():
    file_contents = ""
    prompt_tokens_list = []
    relative_path_list = []
    file_list = list_files(hp.PROJECT_ROOT_PATH, hp.INCLUDED_PATHS, hp.EXCLUDED_PATHS)
    for file_path in file_list :
        relative_path, file_content_str = generate_file_content_string(hp.PROJECT_ROOT_PATH, file_path)
        file_contents += file_content_str
        num_tokens = num_tokens_from_messages(
            messages = [{"role": "system", "name":"file_contents", "content": file_content_str}],
            model = hp.TOKEN_CAL_MODEL_NAME
        )
        prompt_tokens_list.append(num_tokens)
        relative_path_list.append(relative_path)

    messages = hp.messages_sample.copy()
    messages.append({"role": "system", "name":"file_contents", "content": file_contents})
    system_tokens = num_tokens_from_messages(
        messages = messages,
        model = hp.TOKEN_CAL_MODEL_NAME
    )

    df_info = pd.DataFrame()
    df_info['relative_path'] = relative_path_list
    df_info['num_tokens'] = prompt_tokens_list
    df_info = df_info.sort_values('num_tokens', ascending=False)

    return file_contents, system_tokens, df_info


def chat_request(file_contents, prompt_history_queue, user_input, include_file_contents = True, include_prompt_history = True, display_history=False):
    messages = hp.messages_sample.copy()
    
    if include_file_contents :
        file_contents_msg = {"role": "system", "name":"file_contents", "content": file_contents}
        messages.append(file_contents_msg)
    
    if include_prompt_history :
        prompt_history_list = list(prompt_history_queue)
        messages += prompt_history_list
        if display_history :
            for prompt_history in prompt_history_list :
                print(f"{prompt_history['role']} : \n{prompt_history['content']}\n")
    
    user_msg = {"role": "user", "content": user_input}
    messages += [user_msg]
    print("-"*10)
    print(f"user: \n{user_input}\n")
    print("-"*10)
    completion = client.chat.completions.create(
        model=hp.MODEL_NAME,
        messages=messages,
        temperature=0,
        max_tokens=hp.MAX_TOKENS,
    )

    assistant_content = completion.choices[0].message.content
    
    print(f"assistant: \n{assistant_content}\n")
    print(completion.usage)
    
    assistant_msg = {"role": "assistant", "content": assistant_content}
    prompt_history_queue.append(user_msg)
    prompt_history_queue.append(assistant_msg)

    return completion, assistant_content

- Confirm File System

In [3]:
file_contents, system_tokens, df_info = get_project_info()
pprint(('/' + df_info.sort_index()['relative_path']).tolist())

['/.gitignore',
 '/README.md',
 '/app.ipynb',
 '/hparams.py',
 '/hparams.py.sample',
 '/requirements.txt',
 '/utils.py']


# Run

In [6]:
# This is where you can ask questions or give commands to the assistant.
user_input = """

"""

file_contents, system_tokens, df_info = get_project_info() # Call the get_project_info function to retrieve project information.
print(f"system tokens : {system_tokens}") # Print the number of system tokens.
display(df_info.iloc[:10])  # Display information for the top 10 files.

# Call the chat_request function to get the assistant's response to user input.
completion, assistant_content = chat_request(file_contents, prompt_history_queue, user_input[1:][:-1], 
    include_file_contents = True,  # Include file contents in the conversation.
    include_prompt_history = True,  # Include previous conversation history in the conversation.
    display_history = False  # Do not display previous conversation history on the screen.
)

system tokens : 5480


,relative_path,num_tokens
2,app.ipynb,1736
5,requirements.txt,1133
1,README.md,1026
6,utils.py,803
3,hparams.py,375
4,hparams.py.sample,306
0,.gitignore,37


----------
user: 
user_input 위에는

----------
assistant: 
`app.ipynb` 파일에서 `user_input` 변수를 사용하기 전에, 사용자로부터 입력을 받는 부분에 대한 설명을 추가하겠습니다. 아래 코드는 `user_input` 변수 설정 전에 사용자 입력을 설명하는 주석을 포함하고 있습니다.

```python
# Set the user_input variable with the text that you want to send to the assistant.
# This is where you can ask questions or give commands to the assistant.
user_input = """
user_input 위에는
"""
```

이 주석은 `user_input` 변수에 대한 설명을 제공하며, 이 변수에 사용자가 어시스턴트에게 전달하고자 하는 텍스트를 설정하는 부분임을 명시합니다. 여기서는 사용자가 어시스턴트에게 질문을 하거나 명령을 내릴 수 있는 텍스트를 입력합니다.

CompletionUsage(completion_tokens=219, prompt_tokens=6201, total_tokens=6420)
